In [ ]:
%load_ext autoreload

In [ ]:
import asyncio
from pathlib import Path
from typing import Literal

from dspy import ChainOfThought, InputField, OutputField, Signature
from IPython.display import Markdown, display
from langchain.tools import tool
from langchain_core.documents import Document
from langchain_tavily import TavilySearch
from langsmith import traceable
from loguru import logger
from pydantic import BaseModel, Field

from config import config
from data_processing import (
    load_and_process_csv,
    load_documents_from_csv,
    split_documents,
)
from embeddings import initialize_embeddings
from evaluation import (
    RAGEvaluator,
    evaluate_rag_pipeline,
    generate_test_dataset,
)
from rag_chain import RAGOutput
from retrieval import build_retriever
from retrieval_formatter import RetrieverReportGenerator
from source_formatter import SourceFormatter
from utils import print_separator, setup_environment

In [ ]:
# from pubmed_scraper import PubMedScraper

# scraper = PubMedScraper(email = "olandechris@gmail.com")

# df = scraper.search_with_llm(query = "find me 50 papers about covid 19 from 2019 to 2025 by anthony fauci")

2025-10-08 17:17:12.743 | INFO     | pubmed_scraper:parse_query:56 - Parsed query: {'authors': ['Anthony Fauci'], 'topics': ['COVID-19'], 'start_date': '2019/01/01', 'end_date': '2025/12/31', 'max_results': 50, 'filename': 'data/covid_19_anthony_fauci.csv'}
2025-10-08 17:17:12.744 | INFO     | pubmed_scraper:search_with_llm:274 - Searching with: authors=['Anthony Fauci'], topics=['COVID-19']
2025-10-08 17:17:12.747 | INFO     | pubmed_scraper:search_pubmed:86 - Searching: (Anthony Fauci[Author]) AND (COVID-19[Title/Abstract]) AND ("2019/01/01"[Date - Create] : "2025/12/31"[Date - Create])
2025-10-08 17:17:13.970 | INFO     | pubmed_scraper:search_pubmed:91 - Found 25 articles
2025-10-08 17:17:13.976 | INFO     | pubmed_scraper:fetch_details:216 - Fetching batch 1/3
2025-10-08 17:17:17.798 | INFO     | pubmed_scraper:fetch_details:216 - Fetching batch 2/3
2025-10-08 17:17:20.266 | INFO     | pubmed_scraper:fetch_details:216 - Fetching batch 3/3
2025-10-08 17:17:22.150 | INFO     | pubme

In [ ]:
def prepare_data(input_csv: str, output_csv: str | Path | None = None) -> Path:
    """
    Prepare and process the input data.

    Args:
        input_csv: Path to input CSV file
        output_csv: Path to save processed CSV (optional)

    Returns:
        Path to the processed CSV file
    """
    print_separator("DATA PREPARATION")
    if output_csv is None:
        output_csv = Path(config.paths.data_dir) / "tests.csv"

    # Load and process CSV
    print(f"Loading data from {input_csv}")
    df = load_and_process_csv(input_csv, output_csv)

    return Path(output_csv)

In [ ]:
def build_rag_system(csv_path: str):
    """
    Build the complete RAG system.

    Args:
        csv_path: Path to processed CSV file

    Returns:
        Tuple of (rag_chain, splitted_documents, embeddings)
    """
    print_separator("BUILDING RAG SYSTEM")
    print("Initializing Embeddings ...")

    embeddings = initialize_embeddings(
        model_name=config.model.embedding_model,
        cache_dir=config.model.embedding_cache_dir,
    )

    # Load and split the documents
    print("Loading documents...")
    documents = load_documents_from_csv(csv_path)

    print("Splitting documents...")
    splitted_documents = split_documents(documents, embeddings)
    print(f"Created {len(splitted_documents)} document chunks")

    # Build Retriever
    print("Building retriever")
    retriever = build_retriever(splitted_documents, embeddings, config.retriever)

    # Initialize RAG Chain
    print("Initializing RAG Chain ...")

    # Initialize RAG chain
    print("Initializing RAG chain...")
    rag_chain = RAGOutput(
        prompt_name="rlm/rag-prompt",
        retriever=retriever,
        llm_model=config.model.deepseek_model,
    )
    rag_chain.create_chain()

    print("RAG system built successfully")
    return rag_chain, splitted_documents, embeddings

In [ ]:
def run_evaluation(
    rag_chain: RAGOutput,
    test_dataset_path: str | Path | None = None,
    results_path: str | Path | None = None,
):
    """
    Run evaluation on the RAG system.

    Args:
        rag_chain: Configured RAG chain
        test_dataset_path: Path to test dataset (optional)
        results_path: Path to save results (optional)
    """
    print_separator("EVALUATION")

    if test_dataset_path is None:
        test_dataset_path = config.paths.rag_eval_dir / "generated_testset.csv"

    if results_path is None:
        results_path = config.paths.rag_eval_dir / "results_deepseek_fastembed.csv"

    # Run evaluation
    results = evaluate_rag_pipeline(
        rag_chain=rag_chain,
        input_csv_path=str(test_dataset_path),
        output_csv_path=str(results_path),
        question_column="user_input",
    )

    print(f"Evaluation complete. Results shape: {results.shape}")
    return results

In [ ]:
def generate_synthetic_testset(
    splitted_documents, embeddings, llm, testset_size: int = 10
):
    """
    Generate synthetic test dataset using RAGAS.

    Args:
        splitted_documents: Split documents
        embeddings: Embeddings model
        llm: Language model
        testset_size: Number of test samples

    Returns:
        Generated test dataset as DataFrame
    """
    print_separator("GENERATING TEST DATASET")

    output_path = config.paths.rag_eval_dir / "generated_testset.csv"

    dataset = generate_test_dataset(
        documents=splitted_documents,
        embeddings=embeddings,
        llm=llm,
        testset_size=testset_size,
        output_path=str(output_path),
    )

    print(f"Test dataset generated with {len(dataset)} samples")
    return dataset

In [ ]:
def run_full_evaluation(embeddings):
    """
    Run comprehensive evaluation using RAGEvaluator.

    Args:
        embeddings: Embeddings model to use for evaluation

    Returns:
        Dictionary of evaluation results for all model pairs
    """
    print_separator("COMPREHENSIVE EVALUATION")

    # Initialize evaluator
    evaluator = RAGEvaluator(
        max_workers=1,
        timeout=180,
        generative_models=["deepseek-chat"],
        embedding_models=["fastembed"],
    )

    # Run evaluation for all model pairs
    results = evaluator.evaluate_all_models(
        evaluation_embeddings=embeddings, results_dir=str(config.paths.rag_eval_dir)
    )

    # Print summary
    print_separator("EVALUATION SUMMARY")
    for model_pair, df in results.items():
        print(f"\n{model_pair}:")
        print(f"  Shape: {df.shape}")
        if len(df) > 0:
            metric_cols = [
                col
                for col in df.columns
                if col
                not in ["user_input", "reference", "response", "retrieved_contexts"]
            ]
            if metric_cols:
                print(f"  Metrics: {', '.join(metric_cols)}")
                for col in metric_cols:
                    if df[col].dtype in ["float64", "int64"]:
                        print(f"    {col}: {df[col].mean():.4f} (avg)")

    return results

# Program Entry

In [ ]:
# Setup environment
setup_environment()

# Initialize configuration
print_separator("INITIALIZING")
print(f"Data directory: {config.paths.data_dir}")
print(f"RAG evaluation directory: {config.paths.rag_eval_dir}")

llm = config.initialize_llm()
dspy_lm = config.initialize_dspy()
# Prepare data
csv_path = prepare_data("data/gaza_war_impact_children.csv")

# Build RAG system
rag_chain, splitted_documents, embeddings = build_rag_system(str(csv_path))

# Generate test dataset
# generate_synthetic_testset(splitted_documents, embeddings, llm, testset_size=10)

# Run evaluation
# results = run_evaluation(rag_chain)

# eval_results = run_full_evaluation(embeddings)


================================= INITIALIZING =================================

Data directory: data
RAG evaluation directory: RAGEvaluation

=============================== DATA PREPARATION ===============================

Loading data from data/gaza_war_impact_children.csv
Saved processed data to data/tests.csv

============================= BUILDING RAG SYSTEM =============================

Initializing Embeddings ...
Loading documents...
Splitting documents...
Created 24 document chunks
Building retriever
Creating new FAISS index at faiss_index
FAISS index saved to faiss_index
Initializing RAG Chain ...
Initializing RAG chain...
RAG system built successfully


# Iteration II

In [ ]:
csv_path = prepare_data("data/gaza_war_impact_children.csv")
#  Load and split the documents
print("Loading documents...")
documents = load_documents_from_csv(csv_path)

print("Splitting documents...")
splitted_documents = split_documents(documents, embeddings)
print(f"Created {len(splitted_documents)} document chunks")

# Build Retriever
print("Building retriever")
retriever = build_retriever(splitted_documents, embeddings, config.retriever)


=============================== DATA PREPARATION ===============================

Loading data from data/gaza_war_impact_children.csv
Saved processed data to data/tests.csv
Loading documents...
Splitting documents...
Created 24 document chunks
Building retriever
Loading existing FAISS index from faiss_index


In [ ]:
retriever

ContextualCompressionRetriever(base_compressor=DocumentCompressorPipeline(transformers=[EmbeddingsFilter(embeddings=FastEmbed(fe=<fastembed.text.text_embedding.TextEmbedding object at 0x7ff244d52d50>), similarity_fn=<function cosine_similarity at 0x7ff24d367c40>, k=20, similarity_threshold=0.6), EmbeddingsRedundantFilter(embeddings=FastEmbed(fe=<fastembed.text.text_embedding.TextEmbedding object at 0x7ff244d52d50>), similarity_fn=<function cosine_similarity at 0x7ff24d367c40>, similarity_threshold=0.95), FastEmbedRerank(model_name='Xenova/ms-marco-miniLM-L-6-v2', cache_dir='~/.cache/fastembed', top_n=5, encoder=<fastembed.rerank.cross_encoder.text_cross_encoder.TextCrossEncoder object at 0x7ff23fd0e490>)]), base_retriever=EnsembleRetriever(retrievers=[BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7ff23fd0e210>, k=15), VectorStoreRetriever(tags=['FAISS', 'FastEmbed'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ff23fd0ccd0>, search_kwargs={'k': 15}

In [ ]:
def deduplicate_documents(documents: list[list[Document]]):
    seen = set()
    unique_docs = []

    for doc_list in documents:
        for doc in doc_list:
            if doc.page_content not in seen:
                seen.add(doc.page_content)
                unique_docs.append(doc)

    return unique_docs

In [ ]:
class MultiQueryGenerator(Signature):
    """
    Generate multiple semantically diverse versions of a query to improve retrieval.

    Purpose:
        - Rephrase the input query into several natural language variants
        - Maintain the original meaning while exploring different semantic angles
        - Optimize for both vector-based and keyword-based retrieval systems

    Guidelines:
        - Ensure queries are diverse (avoid redundancy)
        - Keep phrasing natural and conversational
        - Highlight key concepts most relevant to the original question
        - Do not change the query’s core intent

    Example:
        Input: "Impact of Gaza war on children"
        Output:
            - "Effects of Gaza conflict on child health and education"
            - "Psychological trauma among Palestinian youth in Gaza"
            - "Humanitarian impact of war on Gaza’s children"
    """

    question = InputField(
        desc="Original user query to expand into multiple rephrased variants"
    )
    num_queries: int = InputField(
        desc="Number of distinct queries to generate (recommended 3–5)"
    )

    search_queries: list[str] = OutputField(
        desc="List of semantically diverse queries that preserve the original intent"
    )

In [ ]:
query_optimizer = ChainOfThought(MultiQueryGenerator)
response = query_optimizer(
    question="Kurt Cobain is the best grunge artist ever", num_queries=3
)
response.search_queries

['Kurt Cobain influence and legacy in grunge music',
 'Why is Kurt Cobain considered the top grunge artist',
 "Kurt Cobain's impact compared to other grunge musicians"]

In [ ]:
dspy_lm.inspect_history()





[2025-10-08T17:37:41.713038]

System message:

Your input fields are:
1. `question` (str): Original user query to expand into multiple rephrased variants
2. `num_queries` (int): Number of distinct queries to generate (recommended 3–5)
Your output fields are:
1. `reasoning` (str): 
2. `search_queries` (list[str]): List of semantically diverse queries that preserve the original intent
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## num_queries ## ]]
{num_queries}

[[ ## reasoning ## ]]
{reasoning}

[[ ## search_queries ## ]]
{search_queries}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Generate multiple semantically diverse versions of a query to improve retrieval.
        
        Purpose:
            - Rephrase the input query into several natu

In [ ]:
@traceable(name="Multiquery Generator")
def generate_queries(question: str, num_queries: int = 2):
    if not question.strip():
        logger.error("Empty query provided")
        return [question]

    try:
        query_optimizer = ChainOfThought(MultiQueryGenerator)
        return query_optimizer(
            question=question, num_queries=num_queries
        ).search_queries
    except Exception as e:
        logger.error(f"An error occurred during query generation: {str(e)}")
        return [question]


generate_queries(question="Impact of the Gaza war on children")

['Humanitarian and health effects of Gaza conflict on children',
 'Psychological trauma and mental health impact on children in Gaza war']

In [ ]:
@tool
async def retriever_tool(search_query: str):
    """
    Retrieves pubmed data using the provided query and generates a report in markdown format.
    """
    report_gen = RetrieverReportGenerator()
    search_queries = generate_queries(question=search_query)

    tasks = [retriever.ainvoke(q) for q in search_queries]
    results = await asyncio.gather(*tasks)

    valid_results = [r for r in results if r]

    if not valid_results:
        return "No relevant documents found for the given query"

    deduplicated_results = deduplicate_documents(valid_results)
    report = report_gen.create_report(deduplicated_results)
    return report["markdown"]

In [ ]:
report = asyncio.run(
    retriever_tool.ainvoke("Impact of the Gaza war on children mental wellbeing")
)

In [ ]:
display(Markdown(report))

# Research Findings Report
*Generated on October 08, 2025 at 06:13 PM*

**Summary:**
- Total unique documents: 5
- Total chunks processed: 7

## Report Statistics

- **Total Documents:** 5
- **Total Chunks:** 7

### Documents with Multiple Chunks:
- https://www.ncbi.nlm.nih.gov/pubmed/22491534: 2 chunks
- https://www.ncbi.nlm.nih.gov/pubmed/30702385: 2 chunks

---

## 1. Quality of life, primary traumatisation, and positive and negative affects in primary school students in the Gaza Strip.
**Authors:** Veronese Guido, Pepe Alessandro, Almurnak Feda, Jaradah Alaa, Hamdouna Husam  
**Publication Date:** 2018-Feb-21  
**URL:** https://www.ncbi.nlm.nih.gov/pubmed/29553411  
**References:** Not available  
**Chunks found:** 1

### Content:

**Relevance Score:** 0.9987984423546961

Article: Quality of life, primary traumatisation, and positive and negative affects in primary school students in the Gaza Strip.Many researchers have reported that exposure to war and ongoing political violence increases mental health problems in children. Results of studies have also shown a high prevalence (58-80%) of post-traumatic stress disorder in war-affected children living in the occupied Palestinian territory. The aim of this study was to estimate the direct and indirect effects of perceived life satisfaction on the consequences of children's exposure to trauma and the balance of positive and negative affect. Palestinian children were recruited from primary schools in four refugee camps in the Gaza Strip (Bureij, Gaza Beach Camp, Jabalia, Rafah). All children had been involved in or witnessed one or more episodes of violence involving other people in the 2 months prior to the study (the 2012 Gaza War). We used the Multidimensional Students Life Satisfaction Scale (peers, self, living environment, school, family), the Positive and Negative Affect Scale for Children, and the revised Children Impact of Events scale (intrusion and avoidance symptoms) to test (through structural equation modelling) the moderation effect of life satisfaction on war trauma via positive emotions. 1276 Palestinian children were enrolled in this study.


## 2. The impact of occupation on child health in a Palestinian refugee camp.
**Authors:** Basak Polly  
**Publication Date:** 2012-Dec  
**URL:** https://www.ncbi.nlm.nih.gov/pubmed/22491534  
**References:** Not available  
**Chunks found:** 2

### Content:

**Chunk 2 (Score: 0.9975309802156953):**

Interviewees revealed that their perceptions of their children's health were determined by the camp's conditions, the current economic climate, past and current political conflict and financial and social restrictions. The understanding of being healthy incorporated physical and mental health as well as social well-being. As a result, 70% of interviewees deemed that their children were not in good health. This finding accelerated to 100% after the Gaza War, showing the negative effect war has on health perceptions. Findings showed that perceptions of physical health are very much interlinked with mental well-being and parents' perceptions of their children's health, and are closely related to their state of mental health. Consequently, a clear correlation can be discerned between the ongoing occupation and its detrimental effects on mental health. Therapeutic and preventive health programmes such as child therapy and stress management that have already been implemented by the Gaza Community Mental Health Programme would be highly beneficial to both children and adults in Dheisheh refugee camp.

---

**Chunk 3 (Score: 0.9898092055332336):**

Article: The impact of occupation on child health in a Palestinian refugee camp.This article focuses on child health in the Palestinian refugee camp of Dheisheh in the West Bank region of the Occupied Palestinian Territories. Thirty in-depth interviews were carried out with parents to determine their perceptions of their children's health. The questions related to physical, mental and social well-being, access to health facilities, factors that were likely to hinder health and measures that could be implemented to improve child health. The study was carried out prior to and during the Gaza War in December 2008 that resulted in the deaths of 1380 Palestinians including 431 children and 112 women [1]. The effects of occupation, conflict and being a refugee had a detrimental impact on perceptions of health.

---


## 3. Is silence about trauma harmful for children? Transgenerational communication in Palestinian families.
**Authors:** Dalgaard Nina Thorup, Diab Safwat Y, Montgomery Edith, Qouta Samir R, Punamäki Raija-Leena  
**Publication Date:** 2019-Apr  
**URL:** https://www.ncbi.nlm.nih.gov/pubmed/30702385  
**References:** Not available  
**Chunks found:** 2

### Content:

**Chunk 4 (Score: 0.9288922754781366):**

Article: Is silence about trauma harmful for children? Transgenerational communication in Palestinian families.Style of family communication is considered important in the transgenerational transmission of trauma. This study had three aims: first, to identify the contents of family communication about past national trauma; second, to examine how parents' current war trauma is associated with transgenerational communication; and third, to analyze the associations between transgenerational communication and children's mental health, measured as posttraumatic stress disorder (PTSD), depression and psychological distress. The study sample consisted of 170 Palestinian families in Gaza Strip, in which both mothers (n = 170) and fathers (n = 170) participated, each with their 11-13-year-old child.

---

**Chunk 7 (Score: 0.9471660811920559):**

Mothers and fathers responded separately to three questions: 1) what did their own parents tell them about the War of 1948, Nakba?; 2) what did they tell their own children about the Nakba?; and 3) What did they tell their own children about the 1967 Arab-Israeli War and military occupation? Current war trauma, as reported separately by mothers, fathers and their children, refers to the Gaza War 2008/09. Children reported their symptoms of PTSD, depression, and psychological distress. Results revealed seven communication content categories and one category indicating maintaining silence about the traumas. Fathers' high exposure to current war trauma was associated with a higher level of communicating facts, reasons, and meanings regarding the1948 and 1967 wars, and mothers' high exposure to current war trauma was associated with a lower level of maintaining silence. Family communication about facts, reasons, and meanings was significantly associated with children not showing PTSD and marginally with not showing psychological distress, while maintaining silence was not associated with children's mental health.

---


## 4. Post-traumatic stress in war veterans and secondary traumatic stress among parents of war veterans five years after the 2014 Israel-Gaza military conflict.
**Authors:** Leshem Shahaf, Keha Eldad, Kalanthroff Eyal  
**Publication Date:** 2023  
**URL:** https://www.ncbi.nlm.nih.gov/pubmed/37497596  
**References:** American Psychiatric Association . (2013). Diagnostic and statistical manual of mental disorders (5th ed.). Washington, DC: American Psychiatric Publishing. | Canfield, J. (2005). Secondary traumatization, burnout, and vicarious traumatization: A review of the literature as it relates to therapists who treat trauma. Smith College Studies in Social Work, 75(2), 81–101. 10.1300/J497v75n02_06 | Dagan, S. W., Ben-Porat, A., & Itzhaky, H. (2016). Child protection workers dealing with child abuse: The contribution of personal, social and organizational resources to secondary traumatization. Child Abuse & Neglect, 51, 203–211. 10.1016/j.chiabu.2015.10.008 https://pubmed.ncbi.nlm.nih.gov/26549769 | David, D., Woodward, C., Esquenazi, J., & Mellman, T. A. (2004). Comparison of comorbid physical illnesses among veterans with PTSD and veterans with alcohol dependence. Psychiatric Services, 55(1), 82–85. 10.1176/appi.ps.55.1.82 https://pubmed.ncbi.nlm.nih.gov/14699207 | Dinshtein, Y., Dekel, R., & Polliack, M. (2011). Secondary traumatization among adult children of PTSD veterans: The role of mother–child relationships. Journal of Family Social Work, 14(2), 109–124. 10.1080/10522158.2011.544021  
**Chunks found:** 1

### Content:

**Relevance Score:** 0.8782210532500938

Article: Post-traumatic stress in war veterans and secondary traumatic stress among parents of war veterans five years after the 2014 Israel-Gaza military conflict.Secondary traumatic stress (STS) has been studied in therapists, spouses, and mainly in children of traumatised individuals. To date, the relationship between children's posttraumatic stress disorder (PTSD) symptoms and their parents' mental health outcomes have been understudied, and specifically, long term STS symptoms of parents of war veterans were not investigated. The current study examined PTSD symptoms among veterans of the 2014 Israel-Gaza war and STS among their parents, five years after the war. The control group consisted of veterans from combat military units who were on active duty at the time of the war but did not actively participate in the war (stand-by units) - providing a natural experiment condition.


## 5. War trauma and infant motor, cognitive, and socioemotional development: Maternal mental health and dyadic interaction as explanatory processes.
**Authors:** Qouta Samir R, Vänskä Mervi, Diab Safwat Y, Punamäki Raija-Leena  
**Publication Date:** 2021-May  
**URL:** https://www.ncbi.nlm.nih.gov/pubmed/33588286  
**References:** Not available  
**Chunks found:** 1

### Content:

**Relevance Score:** 0.967673720948716

Article: War trauma and infant motor, cognitive, and socioemotional development: Maternal mental health and dyadic interaction as explanatory processes.Taking care of infants in conditions of war is highly demanding and a few studies reveal the negative impact of war trauma on maternal and infant well-being. Yet, little is known regarding the influence of trauma on infant development and the potential explanatory mechanisms. First, the present study examines how mothers' prenatal exposure to traumatic war events is associated with infant cognitive, motor, and socioemotional development. Second, it analyses the mediating roles of maternal postpartum mental health problems, quality of dyadic mother-infant interaction, and earlier infant development (at six months) in the association between prenatal traumatic war events and infants' developmental skills at 18 months. This prospective three-wave study involved 502 Palestinian pregnant females in their first trimester during the 2014 Gaza War and participated at delivery (T1) and when the child was six (T2;N = 392) and eighteen (T3; N = 386) months of age. Mothers reported their exposure to traumatic war events (human and material losses, horrors, and threat to life) at T1 and T2, and researchers photo-documented the extent of destruction at T1.




In [ ]:
async def tavily_search_async(
    search_query: str,
    max_results: int = 2,
    num_queries: int = 2,
    include_raw_content: bool = False,
    topic: Literal["general", "news", "finance"] = "general",
):
    if not search_query.strip():
        logger.warning("The search query cannot be an empty string")

    try:
        tavily_search = TavilySearch(
            max_resutls=max_results,
            num_queries=num_queries,
            include_raw_content=include_raw_content,
            topic=topic,
        )
        # Generate multiple queries from the single query
        try:
            search_queries = generate_queries(
                question=search_query, num_queries=num_queries
            )
        except Exception as e:
            logger.warning(
                f"Failed to generate query variations, using original query: {str(e)}"
            )
            search_queries = [search_query]
        # Execute the searches concurrently
        tasks = [tavily_search.ainvoke({"query": q}) for q in search_queries]
        results = await asyncio.gather(*tasks)
        return results
    except Exception as e:
        logger.error(f"An error occurred during web search: {str(e)}")
        return []


@tool
async def web_search(
    search_query: str,
    max_results: int = 1,
    include_raw_content: bool = True,
    markdown_output: bool = False,
):
    """
    Search the web for the given search queries.
    """
    try:
        formatter = SourceFormatter(markdown_output=markdown_output)
        search_response = await tavily_search_async(
            search_query,
            max_results=max_results,
            include_raw_content=include_raw_content,
        )
        deduplicated_response = formatter.deduplicate_and_format_sources(
            search_response
        )
        return deduplicated_response
    except Exception as e:
        logger.error(f"An error occurred during web search: {str(e)}")
        return []

In [ ]:
web_response = asyncio.run(
    web_search.ainvoke("Impact of the Gaza war on children's mental wellbeing")
)

2025-10-08 18:32:01 | INFO     | source_formatter:deduplicate_and_format_sources:124 - Processing sources for deduplication and formatting
2025-10-08 18:32:01 | INFO     | source_formatter:deduplicate_and_format_sources:134 - Found 9 unique sources


In [ ]:
Markdown(web_response)

Sources:

Source 1: New Study: Gaza's Children Face Severe Psychological Toll Amid ...

URL: https://www.warchild.net/news/new-study-gaza-children-psychological-toll/

Most relevant content: A new report out of Gaza lays bare the profound psychological impact of the ongoing war, particularly on children who are injured, disabled, separated from their families, or unaccompanied. The study, *Needs Assessment Study of Children with Disabilities, Injured and Separated or Unaccompanied*, was conducted by the Community Training Centre for Crisis Management (CTCCM) with support from the War Child Alliance. The mental health of Gaza’s children is under constant attack,” says Rob Williams, War Child Alliance CEO. Recognising the acute mental health crisis, War Child, CTCCM and other partners have begun addressing the trauma experienced by more than 17,000 children. War Child is also addressing the needs of unaccompanied and separated children, a particularly vulnerable group.

Full source content (limited to ~1250 tokens):

A new report out of Gaza lays bare the profound psychological impact of the ongoing war, particularly on children who are injured, disabled, separated from their families, or unaccompanied.

[See the Needs Assessment Summary](/documents/339/20241205_WCA_OPT_Gaza_NAS_FINAL.pdf)

The study, *Needs Assessment Study of Children with Disabilities, Injured and Separated or Unaccompanied*, was conducted by the Community Training Centre for Crisis Management (CTCCM) with support from the War Child Alliance. It paints a harrowing picture of children’s mental health under Israeli bombardment and blockade.

“We met with injured, separated, and disabled children and their caregivers to hear from them about the toll of war on their lives. What they shared was devastating – but sadly, not surprising. This study reinforces what we have seen, heard and witnessed over more than a year. Children are traumatised by this war, and we must respond,” says a spokesperson and Project Technical Coordinator from CTCCM in Gaza.

The findings in this study are stark. Caregivers report that 96% of children feel death is imminent, and nearly half believe they will die because of the war. Many children exhibit symptoms of aggression, fear, withdrawal, and severe anxiety, alongside a pervasive sense of hopelessness. Years of displacement, loss, and relentless bombing have left children psychologically scarred and their families in dire circumstances.

The survey of 504 households reveals that 88% of families have been displaced multiple times, with 21% forced to move six or more times. Most families live on less than €122 a month, grappling with soaring prices for food and essentials due to the ongoing blockade and restrictions on humanitarian aid. Meanwhile, 80% of breadwinners are unemployed, reflecting the devastating economic consequences of the war.

“For children in Gaza, nowhere is safe. They have seen homes destroyed, loved ones killed, and schools turned to rubble. Even so-called evacuation zones are not spared from bombing. The mental health of Gaza’s children is under constant attack,” says Rob Williams, War Child Alliance CEO.

The findings of the report\* emphasise that the international community must act urgently. A ceasefire is imperative to allow the unrestricted delivery of humanitarian aid. Critical supplies, including medical provisions and nutritional supplements, must reach those in need immediately. Mental health and psychosocial support must be scaled up to address the widespread trauma experienced by children and their families.

Amidst this devastation, War Child and its partners have stepped in to provide life-saving assistance. Since the war began, their efforts have reached over 180,000 people, including 118,000 children. This includes emergency provisions such as food, clean water, hygiene kits, warm clothing, and blankets.

Recognising the acute mental health crisis, War Child, CTCCM and other partners have begun addressing the trauma experienced by more than 17,000 children. Programmes such as psychological first aid, adapted versions of TeamUp, and recreational activities in child-friendly spaces are providing moments of relief amidst the chaos. These interventions are vital, helping children process their emotions and find resilience in the face of unimaginable stress. War Child is also addressing the needs of unaccompanied and separated children, a particularly vulnerable group. So far, over 580 of these children have been directly supported, with efforts underway to connect them with essential services and caregivers.

*\*Full study by CTCCM available on request.*

## Read more

[verhaal
10/12/2018

Joint Statement: Open Call for a Ceasefire in Gaza, Lebanon and Israel and End to Impunity Amid
a Spiraling Humanitarian Catastrophe and Escalating Regional Conflict](https://www.warchild.net/news/Joint-statement-Open-Call-Ceasefire-Gaza-Lebanon-Israel/)

[verhaal
10/12/2018

Gaza: For 365 Days Too Long, 1.1 Million Children’s Lives Have Been Allowed to Become a War Zone](https://www.warchild.net/news/Gaza-365-days-too-long/)

### Cookie Preferences

Reject All
Save Preferences
Accept All

### Our Cookie Policy

Your privacy matters
==================================================

Source 2: Cases of trauma due to war and violence among children in Gaza

URL: https://www.emro.who.int/emhj-volume-31-2025/volume-31-issue-4/cases-of-trauma-due-to-war-and-violence-among-children-in-gaza.html

Most relevant content: Background: The conflict in Gaza has exposed children to continuous cycles of violence and trauma, profoundly affecting their mental health. Aim: To illustrate

Full source content (limited to ~1250 tokens):

#

* Main
* Search
* [Contact](http://www.emro.who.int/contact-us.html)

* [YouTube](https://www.youtube.com/channel/UCT7a_fVlSrjOs9jyvtH-uhA)
* [Rss feeds](http://www.emro.who.int/rss-feeds.html)
* [Twitter](https://twitter.com/whoemro)
* [Facebook](https://www.facebook.com/WHOEMRO?sk=wall)




* [Home](https://www.emro.who.int)
* [Health topics](/health-topics.html)
* [Data and statistics](/entity/statistics/statistics.html)
* [Media centre](/entity/media/introduction.html)
* [Information resources](/e-library/index.html)
* [Countries](/countries.html)
* [Programmes](/programmes.html)
* [About Us](/entity/about-us/index.html)




### Section menu

#### You are here

* [Eastern Mediterranean Health Journal](/emh-journal/eastern-mediterranean-health-journal/home.html)
  + [About the journal](/emh-journal/eastern-mediterranean-health-journal/about-the-journal.html)
  + [All issues](/emh-journal/past-issues/past-issues.html)
    - [Volume 31 2025](/emhj-volume-31-2025/index.html)
    - [Volume 30 2024](/emhj-volume-30-2024/index.html)
    - [Volume 29 2023](/emhj-volume-29-2023/index.html)
    - [Volume 28 2022](/emhj-volume-28-2022/index.html)
    - [Volume 27 2021](/emhj-volume-27-2021/index.html)
    - [Volume 26 2020](/emhj-volume-26-2020/index.html)
    - [Volume 25, 2019](/emhj-volume-25-2019/index.html)
    - [Volume 24, 2018](/emhj-volume-24-2018/index.html)
    - [Volume 23, 2017](/emhj-volume-23-2017/index.html)
    - [Volume 22, 2016](/emhj-volume-22-2016/index.html)
    - [Volume 21, 2015](/emhj-volume-21-2015/index.html)
    - [Volume 20, 2014](/emhj-vol-20-2014/index.html)
    - [Volume 19, 2013](/emhj-vol-19-2013/)
    - [Volume 18, 2012](/emhj-volume-18-2012/)
    - [Volume 17, 2011](/emhj-volume-17/)
    - [Volume 16, 2010](/emhj-volume-16-2010/)
    - [Volume 15, 2009](/emhj-volume-15-2009/index.html)
    - [Volume 14, 2008](/emhj-volume-14-2008/index.html)
    - [Volume 13, 2007](/emhj-list/emhj-volume-13-2007/)
    - [Volume 12, 2006](/emhj-volume-12-2006/index.html)
    - [Volume 11, 2005](/emhj-list/emhj-volume-11-2005/)
    - [Volume 10, 2004](/emhj-list/emhj-volume-10-2004/)
    - [Volume 9, 2003](/emhj-list/emhj-volume-9-2003/)
    - [Volume 8, 2002](/emhj-list/emhj-volume-8-2002/)
    - [Volume 7, 2001](/emhj-list/emhj-volume-7-2001/)
    - [Volume 6, 2000](/emhj-list/emhj-volume-6-2000/)
    - [Volume 5, 1999](/emhj-list/emhj-volume-5-1999/)
    - [Volume 4, 1998](/emhj-list/emhj-volume-4-1998/)
    - [Volume 3, 1997](/emhj-volume-3-1997/index.html)
    - [Volume 2, 1996](/emhj-volume-2-1996/index.html)
    - [Volume 1, 1995](/emhj-volume-1-1995/index.html)
  + [Information for authors](/emh-journal/authors/)
  + [Information for reviewers](/emh-journal/reviewers/)
  + [Articles in press](/in-press/index.html)
  + [Register for email alert/RSS](/emh-journal/subscribe/)
  + [How to order print copies](/emh-journal/order/)
  + [Disclaimer/copyright](/emh-journal/disclaimer/)
  + [Information resources](/emh-journal/information-resources/index.html)
  + [Contact us](/emh-journal/eastern-mediterranean-health-journal/contact-us.html)
  + [Home](/emh-journal/index.html)




## Cases of trauma due to war and violence among children in Gaza

[PDF version](https://applications.emro.who.int/EMHJ/V31/04/1020-3397-2025-3104-210-215-eng.pdf "EMHJ - Volume 31 - Issue 04 - Cases of trauma due to war and violence among children in Gaza")

Case report

Shehreen Latif1, Enas Abdelraof A Muaileq1, Hani Chaabo1, Fatima Rashid1, Mairead Kelly1, Nida Khatri1, Hannah Alice King1 and Neil M Fournier2

1Children Not Numbers, 124 City Road, London, United Kingdom (Correspondence to Shehreen Latif:
[LatifPsychology@gmail.com](mailto:LatifPsychology@gmail.com)This e-mail address is being protected from spambots. You need JavaScript enabled to view it
). 2Department of Psychology, Trent University, Peterborough ON, Canada.

### Abstract

**Background**: The conflict in Gaza has exposed children to continuous cycles of violence and trauma, profoundly affecting their mental health.

**Aim**: To illustrate the psychological burden on Gaza’s children and highlight their complex mental health needs amid the conflict.

**Methods**: We report cases of 4 children receiving support from Children Not Numbers, who have lived through the violence in Gaza, illustrating the devastating impact of the conflict on their mental health.

**Results**: The children exhibited symptoms of trauma, including emotional dysregulation, social withdrawal, grief, and worsening of pre-existing conditions. Common factors were displacement, loss of family members, physical injuries, and lack of mental health care resources.

**Conclusion**: The cases highlight the severe psychological impact of the conflict on children and the urgent need for interventions to address the crisis and provide mental health services to prevent long-term consequences for the wellbeing of children in Gaza.

Keywords: war, conflict, violence, displacement, trauma,... [content truncated]
==================================================

Source 3: Impact of trauma on Palestinian children's mental health: lessons ...

URL: https://pmc.ncbi.nlm.nih.gov/articles/PMC6735244/

Most relevant content: Impact of trauma on Palestinian children’s mental health: lessons from the Gaza studies - PMC  Our initial study, after the end of the first intifada (uprising), established a high prevalence rate (41%) of post-traumatic stress reactions among children (aged 6– 11 years) and their significant association with traumatic events experienced by the children, as well as with other behavioural and emotional problems (Thabet & Vostanis, 1999 _a_). In order to understand the underlying mechanisms operating between trauma and mental health problems, in a subsequent study we compared children whose houses had been demolished with children matched for age and gender from areas indirectly exposed to other types of traumatic events, mainly through the media and adults (Thabet _et al_, 2002).

Full source content (limited to ~1250 tokens):

Impact of trauma on Palestinian children’s mental health: lessons from the Gaza studies - PMC 

===============
[Skip to main content](https://pmc.ncbi.nlm.nih.gov/articles/PMC6735244/#main-content)

![Image 1](https://pmc.ncbi.nlm.nih.gov/static/img/us_flag.svg)

An official website of the United States government

Here's how you know

Here's how you know

![Image 2](https://pmc.ncbi.nlm.nih.gov/static/img/icon-dot-gov.svg)

**Official websites use .gov**

 A **.gov** website belongs to an official government organization in the United States.

![Image 3](https://pmc.ncbi.nlm.nih.gov/static/img/icon-https.svg)

**Secure .gov websites use HTTPS**

 A **lock** ( ) or **https://** means you've safely connected to the .gov website. Share sensitive information only on official, secure websites.

[![Image 4: NCBI home page](https://pmc.ncbi.nlm.nih.gov/static/img/ncbi-logos/nih-nlm-ncbi--white.svg)](https://www.ncbi.nlm.nih.gov/)

 Search 

Log in
*   [Dashboard](https://www.ncbi.nlm.nih.gov/myncbi/)
*   [Publications](https://www.ncbi.nlm.nih.gov/myncbi/collections/bibliography/)
*   [Account settings](https://www.ncbi.nlm.nih.gov/account/settings/)
*    Log out 

 Search…   Search NCBI 

Primary site navigation

![Image 5: Close](https://pmc.ncbi.nlm.nih.gov/static/img/usa-icons/close.svg) Search  ![Image 6: Search](blob:http://localhost/d03881a649d411fb287ca34fa7ea6fc4)

Logged in as:

*   [Dashboard](https://www.ncbi.nlm.nih.gov/myncbi/)
*   [Publications](https://www.ncbi.nlm.nih.gov/myncbi/collections/bibliography/)
*   [Account settings](https://www.ncbi.nlm.nih.gov/account/settings/)

Log in

[](https://pmc.ncbi.nlm.nih.gov/ "Home")

Search PMC Full-Text Archive 

Search in PMC![Image 7: Search](https://pmc.ncbi.nlm.nih.gov/static/img/usa-icons-bg/search--white.svg)

*   [Journal List](https://pmc.ncbi.nlm.nih.gov/journals/)
*   [User Guide](https://pmc.ncbi.nlm.nih.gov/about/userguide/)

*   [](https://pmc.ncbi.nlm.nih.gov/articles/PMC6735244/pdf/IP-1-05.pdf)Download PDF
*   Add to Collections
*   Cite
*   Permalink PERMALINK
---------

Copy   

 As a library, NLM provides access to scientific literature. Inclusion in an NLM database does not imply endorsement of, or agreement with, the contents by NLM or the National Institutes of Health.

 Learn more: [PMC Disclaimer](https://pmc.ncbi.nlm.nih.gov/about/disclaimer/) | [PMC Copyright Notice](https://pmc.ncbi.nlm.nih.gov/about/copyright/)

![Image 8: International Psychiatry logo](https://cdn.ncbi.nlm.nih.gov/pmc/banners/logo-bjpsychint.png)

Int Psychiatry

. 2003 Oct 1;1(2):5–6.

*   [Search in PMC](https://pmc.ncbi.nlm.nih.gov/search/?term=%22Int%20Psychiatry%22%5Bjour%5D)
*   [Search in PubMed](https://pubmed.ncbi.nlm.nih.gov/?term=%22Int%20Psychiatry%22%5Bjour%5D)
*   [View in NLM Catalog](https://www.ncbi.nlm.nih.gov/nlmcatalog?term=%22Int%20Psychiatry%22%5BTitle%20Abbreviation%5D)
*   [Add to search](https://pmc.ncbi.nlm.nih.gov/articles/PMC6735244/?term=%22Int%20Psychiatry%22%5Bjour%5D)

Impact of trauma on Palestinian children’s mental health: lessons from the Gaza studies
=======================================================================================

[Panos Vostanis](https://pubmed.ncbi.nlm.nih.gov/?term=%22Vostanis%20P%22%5BAuthor%5D)

### Panos Vostanis

1 Professor of Child and Adolescent Psychiatry, University of Leicester, Westcotes House, Westcotes Drive, Leicester LE3 0QU, UK, email pv11@le.ac.uk

Find articles by [Panos Vostanis](https://pubmed.ncbi.nlm.nih.gov/?term=%22Vostanis%20P%22%5BAuthor%5D)

1

*   Author information
*   Article notes
*   Copyright and License information

1 Professor of Child and Adolescent Psychiatry, University of Leicester, Westcotes House, Westcotes Drive, Leicester LE3 0QU, UK, email pv11@le.ac.uk

Collection date 2003 Oct.

© 2003 The Royal College of Psychiatrists

This is an open access article distributed under the terms of the Creative Commons Non-Commercial, No Derivatives (CC BY-NC-ND) license ([http://creativecommons.org/licenses/by-nc-nd/4.0/](http://creativecommons.org/licenses/by-nc-nd/4.0/)).

[PMC Copyright notice](https://pmc.ncbi.nlm.nih.gov/about/copyright/)

PMCID: PMC6735244 PMID: [31507660](https://pubmed.ncbi.nlm.nih.gov/31507660/)

[](https://pmc.ncbi.nlm.nih.gov/articles/PMC6735244/)Abstract
-------------------------------------------------------------

Children exposed to violence are at high risk of developing a range of mental health problems, predominantly post-traumatic stress disorder (PTSD) and depression (Yule, [1999](https://pmc.ncbi.nlm.nih.gov/articles/PMC6735244/#r1)). Children in war zones can be affected not only directly but also indirectly, for example through their basic health needs not being met, the loss of family members, disruption of social networks, internal displacement and their parents’ responses.

* * *

Since the late 1990s, we have run a number of studies in the Gaza Strip, with Dr Abdel Aziz Thabet and his research team. Our initial study,... [content truncated]
==================================================

Source 4: Children in Gaza have suffered “relentless mental harm” during five ...

URL: https://www.savethechildren.net/news/complete-psychological-destruction-children-gaza-have-suffered-relentless-mental-harm-during

Most relevant content: **RAMALLAH, 12 MARCH 2024 –** Five months of violence, displacement, starvation and disease on top of nearly 17 years of a blockade have caused relentless mental harm to children in Gaza, Save the Children said today. In the report, mental health and child protection experts working with Save the Children in Gaza said that without urgent action, starting with an immediate, definitive ceasefire and safe, unfettered humanitarian access, the war will inflict further lifelong detrimental mental harm, with rapidly shrinking opportunities to recover. While needs are soaring, the latest escalation in violence and the siege have caused a total collapse of mental health services in Gaza, with the six public community mental health centres and Gaza’s only inpatient psychiatric hospital no longer functioning, Save the Children said.

Full source content (limited to ~1250 tokens):

[Skip to main content](#main-content)

Five months of violence, displacement, starvation and disease on top of nearly 17 years of a blockade have caused relentless mental harm to children in Gaza

**RAMALLAH, 12 MARCH 2024 –** Five months of violence, displacement, starvation and disease on top of nearly 17 years of a blockade have caused relentless mental harm to children in Gaza, [Save the Children said today](https://resourcecentre.savethechildren.net/document/trapped-and-scarred-the-compounding-mental-harm-inflicted-on-palestinian-children-in-gaza/). Parents and caregivers told the child rights organisation that children’s capacity to even imagine a future without war has virtually disappeared.

The emotional distress of dodging bombs and bullets, losing loved ones, being forced to flee through streets littered with debris and corpses, and waking up every morning not knowing if they will be able to eat has also left parents and caregivers increasingly unable to cope. The support, services and tools they need to care for their children are further and further out of reach.

### A humanitarian catastrophe is unfolding in Gaza. Your support is urgently needed.

[DONATE NOW](/donate/children-crisis-donate-child-emergency-fund "DONATE NOW")

In [findings](https://resourcecentre.savethechildren.net/document/trapped-and-scarred-the-compounding-mental-harm-inflicted-on-palestinian-children-in-gaza/) published today, Dalia\*, a mother in Gaza, said: “*Our children have already lived through different wars. They already lacked resilience and now it’s very difficult to cope. The children are scared, angry and can’t stop crying. Even many adults do the same. This is too much for adults to cope with, let alone children.*”

The insights form an addendum to Save the Children [research from 2022](https://www.savethechildren.net/news/after-15-years-blockade-four-out-five-children-gaza-say-they-are-living-depression-grief-and) charting the profound impacts of the Israeli Government’s then 15-years of blockade on the mental health of childreni.

In the report, mental health and child protection experts working with Save the Children in Gaza said that without urgent action, starting with an immediate, definitive ceasefire and safe, unfettered humanitarian access, the war will inflict further lifelong detrimental mental harm, with rapidly shrinking opportunities to recover.

Parents noted that children have given up on their hopes or ambitions for the future.

Samer\*, a father, said: *“One of my sons dreamt of becoming an engineer and the other a policeman. Now one wants to drive a donkey cart – because he sees this reality [...] [My other son’s] dream is selling biscuits in front of the house.”*

Amal\*, a mother of four children in Gaza aged between 7 and 14, said: “*Some of my children can’t concentrate on basic tasks anymore. They forget things that I’ve told them immediately and can’t remember things that have just happened. I wouldn’t even say that their mental health has deteriorated – it’s been obliterated. Complete psychological destruction.”*

The findings come as the Ministry of Health in Gaza reports that more than 30,717 people, including 12,550 children, have been killed since Israel’s military escalation in Gaza that started on 7 October\* in retaliation to attacks on Israel killing 1200 people, including 33 children and taking more than 240 hostages, according to the Government of Israel.

Lack of food and clean water is also creating a catastrophic hunger crisis, with nearly every child in Gaza at risk of famine. At least 15 children have died of malnutrition and dehydration in northern Gaza, according to Gaza’s Ministry of Health\*. With barely any health facilities functioning and families cut off from medical services, it is likely that the real figure is far higher – and all this is causing anxiety and stress in children and families to skyrocket, Save the Children said.

Even before 7 October, children in Gaza were living with exceptionally poor mental health due to cyclical escalations of violence, the impacts of the blockade including restrictions on freedom of movement and access to essential services, economic collapse, and separation from family and friends.

Those surveyed for the new report said they had witnessed a dramatic deterioration in the mental health of children that is much worse than during previous escalations in violence, manifesting in fear, anxiety, disordered eating, bedwetting, hyper-vigilance and sleep problems as well as behavioural changes such as an alternation in attachment style with parents, regression and aggression.

Waseem\*, a father, said: “*Children here have seen everything. They’ve seen the bombs, the deaths, the bodies – we can’t pretend to them any more. Now they understand and have seen everything. Now, my son can even tell what types of explosives are falling – he can hear the difference*.”

While needs are soaring, the latest escalation in violence and the... [content truncated]
==================================================

Source 5: Consequences of war-related traumatic stress among Palestinian ...

URL: https://www.sciencedirect.com/science/article/pii/S2212657023000478

Most relevant content: The mental health of children and young people in the Gaza Strip is a pressing public health concern, particularly PTSD, depression, and anxiety.

Full source content (limited to ~1250 tokens):

Consequences of war-related traumatic stress among Palestinian young people in the Gaza Strip: A scoping review - ScienceDirect

===============

[Skip to main content](https://www.sciencedirect.com/science/article/pii/S2212657023000478#screen-reader-main-content)[Skip to article](https://www.sciencedirect.com/science/article/pii/S2212657023000478#screen-reader-main-title)

[![Image 2: Elsevier logo](https://www.sciencedirect.com/shared-assets/24/images/elsevier-non-solus-new-grey.svg)](https://www.sciencedirect.com/)

*   [Journals & Books](https://www.sciencedirect.com/browse/journals-and-books)

*      
*   [](https://www.sciencedirect.com/search "Search") 

[](https://www.sciencedirect.com/user/login?targetURL=%2Fscience%2Farticle%2Fpii%2FS2212657023000478&from=globalheader "My account")

[](https://www.sciencedirect.com/user/institution/login?targetURL=%2Fscience%2Farticle%2Fpii%2FS2212657023000478 "Sign In")

*   [View**PDF**](https://www.sciencedirect.com/science/article/pii/S2212657023000478/pdfft?md5=5d2c329d4723ea093be0b6a66ab0ab41&pid=1-s2.0-S2212657023000478-main.pdf)
*   Download full issue

Search ScienceDirect 

Outline
-------

1.   [Abstract](https://www.sciencedirect.com/science/article/pii/S2212657023000478#abs0001 "Abstract")
2.   [Keywords](https://www.sciencedirect.com/science/article/pii/S2212657023000478#keys0001 "Keywords")
3.   [1. Background](https://www.sciencedirect.com/science/article/pii/S2212657023000478#sec0004 "1. Background")
4.   [2. Methods](https://www.sciencedirect.com/science/article/pii/S2212657023000478#sec0008 "2. Methods")
5.   [3. Results](https://www.sciencedirect.com/science/article/pii/S2212657023000478#sec0014 "3. Results")
6.   [4. Discussion](https://www.sciencedirect.com/science/article/pii/S2212657023000478#sec0025 "4. Discussion")
7.   [5. Strengths and limitations](https://www.sciencedirect.com/science/article/pii/S2212657023000478#sec0029 "5. Strengths and limitations")
8.   [6. Conclusion](https://www.sciencedirect.com/science/article/pii/S2212657023000478#sec0030 "6. Conclusion")
9.   [Authors’ contributions](https://www.sciencedirect.com/science/article/pii/S2212657023000478#sec0031 "Authors’ contributions")
10.   [Declaration of Competing Interest](https://www.sciencedirect.com/science/article/pii/S2212657023000478#coi0001 "Declaration of Competing Interest")
11.   [Study funding](https://www.sciencedirect.com/science/article/pii/S2212657023000478#sec0033 "Study funding")
12.   [Acknowledgements](https://www.sciencedirect.com/science/article/pii/S2212657023000478#sec0034 "Acknowledgements")
13.   [Appendix. Supplementary materials](https://www.sciencedirect.com/science/article/pii/S2212657023000478#sec0036 "Appendix. Supplementary materials")
14.   [References](https://www.sciencedirect.com/science/article/pii/S2212657023000478#cebibl1 "References")

Show full outline

[Cited by (39)](https://www.sciencedirect.com/science/article/pii/S2212657023000478#section-cited-by)
-----------------------------------------------------------------------------------------------------

Figures (1)
-----------

1.   [![Image 3: Fig. 1. PRISMA flow of information through the different phases of a scoping review](https://ars.els-cdn.com/content/image/1-s2.0-S2212657023000478-gr1.sml)](https://www.sciencedirect.com/science/article/pii/S2212657023000478#fig0001)

Tables (6)
----------

1.   [Table 1](https://www.sciencedirect.com/science/article/pii/S2212657023000478#tbl0001 "First and second search strategy.")
2.   [Table 2](https://www.sciencedirect.com/science/article/pii/S2212657023000478#tbl0002 "Summary of data extraction from the 24 studies.")
3.   [Table 3](https://www.sciencedirect.com/science/article/pii/S2212657023000478#tbl0003 "Mental health outcomes identified by the scoping review.")
4.   [Table 4](https://www.sciencedirect.com/science/article/pii/S2212657023000478#tbl0004 "Types of measurement tools and instruments used in the scoped studies.")
5.   [Table 5.1](https://www.sciencedirect.com/science/article/pii/S2212657023000478#tbl0005 "Knowledge in the literature Palestinian young people's mental health.")
6.   [Table 5.2](https://www.sciencedirect.com/science/article/pii/S2212657023000478#tbl0006 "Gaps in the literature Palestinian young people's mental health.")

Extras (3)
----------

1.   Download all 
2.   [Document](https://www.sciencedirect.com/science/article/pii/S2212657023000478#ecom0001 "Document")
3.   [Document](https://www.sciencedirect.com/science/article/pii/S2212657023000478#ecom0002 "Document")
4.   [Document](https://www.sciencedirect.com/science/article/pii/S2212657023000478#ecom0003 "Document")

[![Image 4: Elsevier](https://www.sciencedirect.com/us-east-1/prod/01a87eb052a33b2e07e2724eec27aba07e34f8bd/image/elsevier-non-solus.svg)](https://www.sciencedirect.com/journal/mental-health-and-prevention "Go to Mental Health & Prevention on ScienceDirect")

[Mental Health &... [content truncated]
==================================================

Source 6: Mental Health Crisis for Children in Gaza | UNICEF USA

URL: https://www.unicefusa.org/stories/mental-health-crisis-children-gaza

Most relevant content: * Share UNICEF USA | <strong>Mental Health Crisis</strong> for Children in Gaza on Facebook (opens in a new tab) * Share UNICEF USA | <strong>Mental Health Crisis</strong> for Children in Gaza on Twitter (opens in a new tab) * Share UNICEF USA | <strong>Mental Health Crisis</strong> for Children in Gaza on LinkedIn (opens in a new tab) * Share UNICEF USA | <strong>Mental Health Crisis</strong> for Children in Gaza on Bluesky (opens in a new tab) UNICEF is on the ground, working with partners to support children's mental health and well-being. By April 2024, as the bombing, displacement and deprivation continued, UNICEF estimated that 100 percent of children in Gaza were in need of **mental health and psychosocial support**.

Full source content (limited to ~1250 tokens):

[Skip to main content](#content)

[Read Stories](/stories)
[Donate Now](?form=donate)
[Give Monthly](/?form=FUNMQCUVJVT)



Emergency Response

# **Mental Health Crisis** for Children in Gaza

July 10, 2025

Sarah Ferguson

Est. Reading Time: 6 mins

* [Share UNICEF USA | <strong>Mental Health Crisis</strong> for Children in Gaza on Facebook (opens in a new tab)](https://www.facebook.com/sharer/sharer.php?u=https%3A%2F%2Fwww.unicefusa.org%2Fstories%2Fmental-health-crisis-children-gaza)
* [Share UNICEF USA | <strong>Mental Health Crisis</strong> for Children in Gaza on Twitter (opens in a new tab)](https://twitter.com/intent/tweet?text=UNICEF+USA+%7C+%3Cstrong%3EMental+Health+Crisis%3C%2Fstrong%3E+for+Children+in+Gaza%0Ahttps%3A%2F%2Fwww.unicefusa.org%2Fstories%2Fmental-health-crisis-children-gaza)
* [Share UNICEF USA | <strong>Mental Health Crisis</strong> for Children in Gaza on LinkedIn (opens in a new tab)](https://www.linkedin.com/sharing/share-offsite/?url=https%3A%2F%2Fwww.unicefusa.org%2Fstories%2Fmental-health-crisis-children-gaza)
* [Share UNICEF USA | <strong>Mental Health Crisis</strong> for Children in Gaza on Bluesky (opens in a new tab)](https://bsky.app/intent/compose?text=UNICEF+USA+%7C+%3Cstrong%3EMental+Health+Crisis%3C%2Fstrong%3E+for+Children+in+Gaza%0Ahttps%3A%2F%2Fwww.unicefusa.org%2Fstories%2Fmental-health-crisis-children-gaza)

After 20 months of relentless bombardment, mass displacement and loss, every child in Gaza craves peace and stability. UNICEF is on the ground, working with partners to support children's mental health and well-being. Here's how you can help.

Ten-year-old Waed remembers vividly the terrifying day she and her family were forced to flee Khan Younis under fire, running past tanks to find shelter in a school. “That memory is stuck in my head,” she says.

Waed speaks softly as she describes a technique she learned at a UNICEF-supported mental health and psychosocial support activity. It's called the “TV method” — she visualizes a difficult memory and then imagines breaking it like a screen.

She tries to use every coping mechanism she’s learned, but the fear she felt on the day her family fled their home is hard to shake.

## UNICEF-supported activities help children cope with the psychological impact of conflict

For almost two years, children in Gaza have opened their eyes every morning to another day of displacement, loss and lack of access to basic services. The threat of violence is always there, the trauma compounding.

In humanitarian tents, children gather to share their fears and sorrows, and to comfort one another. UNICEF-trained counselors share strategies that help kids cope with anxiety, grief and despair. Recreational kits provided by UNICEF contain supplies for stress-relieving games and activities.

Thirteen-year-old Raja learned to visualize a safe space by closing her eyes and imagining herself in her home, now destroyed. Sometimes she pictures herself enjoying a warm meal of rice and meat — something she hasn’t had in over four months due to the [**severe food shortages**](/stories/children-gaza-critical-risk-famine) in Gaza. “It’s a meal we used to eat together as a family,” she said quietly. “Now, I can only imagine it.”

Raja admits the exercise wasn’t easy at first. Revisiting a place she never thought she’d lose is painful. “That home doesn’t exist anymore,” she says, her voice heavy with grief. “And that breaks my heart.”

***Related:*** [***Helping Gaza's Children Cope With Trauma***](/stories/helping-gazas-children-cope-trauma)

## Watch the video: The constant threat of bombardment takes a heavy toll

## Families are fragmented, psychosocial distress is widespread

"We do a lot of trauma work in Gaza, and the professionals there remind me that you don’t call it P.T.S.D. in Gaza because there’s nothing *post* — there’s always new traumas coming," UNICEF spokesperson James Elder recently told [**The New Yorker**](https://www.newyorker.com/news/q-and-a/the-war-on-gazas-children). "They give children skills to deal with nightmares at night."

A girl Elder met described how she’d pretend she was in her grandfather’s garden and try to smell the basil to help her with nightmares. "But as the child psychologist would say, they’re not just nightmares," Elder says. "They’re also a reality."

Thirteen-year-old Raja, center, participates in a UNICEF-supported MHPSS activity in Khan Younis. © UNICEF/UNI821436/Eleyan

## Every child in the Gaza Strip needs mental health support

In October 2023, UNICEF Executive Director Catherine Russell was already sounding the alarm on the devastating physical and emotional harm caused by the ongoing conflict and deepening humanitarian crisis.

"Children in both Israel and the State of Palestine are experiencing terrible trauma — the consequences of which could last a lifetime," Russell said, urging the United Nations Security Council to pass a resolution reminding parties of their obligations under international... [content truncated]
==================================================

Source 7: Mental Health of Children in Gaza - PAMA

URL: https://pamausa.org/mental-health-of-children-in-gaza/

Most relevant content: ## Mental Health of Children in Gaza The ongoing crisis has severely impacted mental health in Gaza of its population, particularly affecting children and young people who make up 65% of residents. PAMA and partner organizations are committed to supporting mental health for Gaza’s children through targeted initiatives. Mental health teams provided vital psychological support to over 1,700 children in Gaza across nine IDP shelters. The mental health crisis facing children in Gaza demands immediate, comprehensive, and compassionate global intervention. Through dedicated teams of healthcare professionals working across IDP shelters, mental health professionals continue to provide crucial mental health support  and therapeutic interventions to thousands of children and families in Gaza.

Full source content (limited to ~1250 tokens):

[Skip to content](#content)


[Donate Now](https://palestinian-ama.networkforgood.com/projects/193650-urgent-gaza-medical-appeal-2023)

[Donate Now](?campaign=604664)

[Donate Now](?campaign=604664)

* [General](https://pamausa.org/category/general/)

## Mental Health of Children in Gaza

The ongoing crisis has severely impacted mental health in Gaza of its population, particularly affecting children and young people who make up 65% of residents. Even before recent events, studies indicated that over half of Gaza’s children experienced PTSD. The current situation, marked by displacement and loss, has intensified psychological trauma among the population of 2.1 million, of whom two-thirds are refugees.

Implications of mental health including anxiety, depression, and PTSD often go untreated due to multiple barriers: limited mental health resources, insufficient training among healthcare providers, and restrictions on humanitarian aid. This lack of treatment can have far-reaching consequences, affecting children’s educational outcomes, social development, and future employment prospects. The psychological effect of war and sense of hopelessness can make young people particularly vulnerable to negative influences, contributing to ongoing cycles of social instability.

### **Psychological Trauma Experienced by Children in Gaza**

The psychological effect of war encompasses a complex web of traumatic experiences. Children in Gaza are exposed to continuous bombing and violence, witnessing the loss of parents and family members. The complete disruption of basic life necessities—including water, education, and safety—compounds their psychological suffering. The widespread destruction of homes, schools, and healthcare facilities has stripped away every sense of security and normalcy.

### **Impact of War on Children’s Mental Health**

A recent needs assessment study conducted by the Community Training Centre for Crisis Management, supported by the War Child Alliance, reveals the impact of war on children. The research, which examined children with disabilities and those who are injured or unaccompanied, found an alarming 96% of children live in constant fear for their lives, with nearly half believing they will not survive the conflict. Children commonly display behavioral changes including aggression, withdrawal, and severe anxiety.

The study of 504 households also highlights the widespread displacement of families, with 88% having been forced to relocate multiple times – over a fifth moving six times or more. The economic toll is equally severe, with 80% of primary income earners now unemployed. Most families survive on less than €122 monthly while facing inflated prices for basic necessities due to restricted access to humanitarian supplies.

### **The Importance of Psychosocial Support and Interventions**

Comprehensive psychosocial support is crucial for preventing long-term mental destruction. It represents more than treatment—it’s a lifeline that helps children rebuild their emotional worlds, restore hope, and recreate a sense of future possibility. By addressing trauma holistically, these interventions aim to support the community’s collective healing. 

The PAMA-sponsored project with Juzoor (Welfare Association) demonstrates its importance through standardized shelter activities that prioritize psychosocial first aid, specifically identifying children at risk, and providing intervention therapy within the shelters. This comprehensive approach resulted in serving 3,071 individuals with mental health and psychosocial support in one month alone, with children making up approximately 60% of beneficiaries. Structured interventions help children cope with the ongoing crisis through group activities, demonstrating the importance of regular, sustained psychological care.

What makes this support particularly crucial is its integration into the broader healthcare system. The report from the implementation team in Gaza shows that psychosocial support is delivered alongside other vital health services through mobile health teams that include doctors, nurses, psychologists, and midwives. This integrated approach ensures that mental health in Gaza support isn’t treated as an afterthought but as a fundamental component of comprehensive healthcare delivery.

### **Gaza Community Mental Health Programs**

PAMA and partner organizations are committed to supporting mental health for Gaza’s children through targeted initiatives. These include child trauma counseling, school-based mental health support, community resilience workshops, and trauma-informed care training for local healthcare providers. 

PAMA sponsored programs by local Palestinian organizations in 2024. Mental health teams provided vital psychological support to over 1,700 children in Gaza across nine IDP shelters. Mental health professionals working within PAMA’s mobile health teams conducted psychological first aid activities and therapeutic interventions... [content truncated]
==================================================

Source 8: [PDF] Mental Health and Psychosocial Support (MHPSS) in Gaza | UNRWA

URL: https://www.unrwa.org/sites/default/files/content/resources/22.8.24_-_mhpss_300_day_report_final.pdf

Most relevant content: From the first days of the war, the mental health impact on children was starkly visible. Since October 2023, the forced closure of schools,

Note: No raw content available for this source
==================================================

Source 9: A narrative review of mental health and psychosocial impact of the ...

URL: https://www.emro.who.int/afg/a-narrative-review-of-mental-health-and-psychosocial-impact-of-the-war-in-gaza.html

Most relevant content: Thabet and Vostanis (3) reported that 54% of children in Gaza had PTSD, with comorbid depression (41%) and anxiety (34%), due to traumatic events like bombings

Note: No raw content available for this source

# Report Generation

In [ ]:
class Section(BaseModel):
    name: str = Field(
        description="Medical research section title reflecting PubMed study focus (e.g., 'Pediatric Mental Health Outcomes', 'Conflict-Related Injuries', 'Public Health Impact')"
    )
    description: str = Field(
        description="Medical research context explaining what peer-reviewed evidence this section will analyze"
    )
    research: bool = Field(
        description="True if requires current PubMed literature search, systematic reviews, or specific clinical studies. False only for background/methodology sections"
    )
    content: str = Field(
        description="Specific medical evidence requirements: study types needed (RCTs, cohort studies, case reports), population focus, clinical outcomes, or epidemiological data"
    )

    audience_complexity: str = Field(
        description="Medical evidence level: 'clinical_practitioners', 'public_health_officials', 'medical_researchers', or 'policy_makers'",
        default="clinical_practitioners",
    )

    estimated_length: str = Field(
        description="Evidence depth: 'brief', 'standard', or 'comprehensive'",
        default="standard",
    )

    dependencies: list[str] = Field(
        description="Sections that must precede this one for proper medical context",
        default_factory=list,
    )

    success_criteria: str = Field(
        description="Specific medical research outcome or evidence synthesis goal",
        default="",
    )


class Sections(BaseModel):
    sections: list[Section] = Field(
        description="Medical literature review sections following systematic review or evidence synthesis structure"
    )

    total_estimated_length: str = Field(
        description="Medical review scope: 'rapid_review', 'systematic_review', or 'comprehensive_meta_analysis'",
        default="systematic_review",
    )

    primary_audience: str = Field(
        description="Target medical audience: 'clinicians', 'public_health_officials', 'researchers', 'humanitarian_workers', or 'policy_makers'",
        default="clinicians",
    )

    narrative_strategy: str = Field(
        description="Medical research approach: 'systematic_review', 'scoping_review', 'epidemiological_analysis', or 'clinical_evidence_synthesis'",
        default="systematic_review",
    )

In [ ]:
structured_llm = llm.with_structured_output(Sections)
result = structured_llm.invoke("Impact of the Gaza war on children")
result

Sections(sections=[Section(name='Pediatric Mortality and Morbidity', description='Analysis of direct conflict-related child deaths, injuries, and trauma in Gaza', research=True, content='Requires epidemiological data on child mortality rates, injury patterns (blast injuries, shrapnel wounds, burns), and trauma statistics from recent conflict periods. Need peer-reviewed studies, hospital records, and humanitarian organization reports.', audience_complexity='medical_researchers', estimated_length='comprehensive', dependencies=[], success_criteria='Quantitative assessment of direct conflict impact on child survival'), Section(name='Mental Health and Psychosocial Impact', description="Evidence on PTSD, anxiety, depression, and developmental trauma in Gaza's pediatric population", research=True, content='Systematic review of mental health studies, PTSD prevalence, trauma-related disorders, and psychosocial interventions. Need longitudinal studies, clinical assessments, and validated screeni